## Preparation

### Libraries and Global Variables

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

random_state = 42

### Importing dataset

In [ ]:
pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
X = wine_quality.data.features 
y = wine_quality.data.targets 
df = pd.concat([X, y], axis=1)



## Exploratory Analysis

### Basic Information

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.head()

### YData Profiling Report

In [ ]:
pip install ydata-profiling

In [ ]:
from ydata_profiling import ProfileReport
ProfileReport(df)

### Box plots 

In [ ]:
%matplotlib inline

features = df.columns[:-1]
n_rows = (len(features) + 1) // 2

plt.figure(figsize=(12, n_rows * 4))
for i, feature in enumerate(features):
    plt.subplot(n_rows, 2, i + 1)
    sns.boxplot(x='quality', y=feature, data=df)
    plt.title(f'{feature} vs Wine Quality')



plt.tight_layout()
plt.show()

### Histogram of the Output Score

In [ ]:
# Histogram of wine quality (target variable)
sns.countplot(x='quality', data=df)
plt.title('Distribution of Wine Quality')
plt.xlabel('Quality')
plt.ylabel('Frequency')
plt.show()

### Histogram of the All Features


In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import matplotlib.pyplot as plt


X = wine_quality.data.features
y = wine_quality.data.targets

df = pd.concat([X, y], axis=1)

plt.figure(figsize=(12, 10))

# Plot histogram for each input feature
df.iloc[:, :-1].hist(bins=20, figsize=(12, 10), layout=(4, 3))  # Exclude the target column

# Add a title
plt.suptitle('Histograms of Input Features', fontsize=16)

# Show the plot
plt.tight_layout()
plt.show()


### Correlation Heatmap of Input Features

In [ ]:
# Calculate correlation matrix
correlation_matrix = df.corr()

# Set up the figure size
plt.figure(figsize=(12, 8))

# Create the heatmap
sns.heatmap(
    correlation_matrix, 
    annot=True,          
    cmap='coolwarm',     
    fmt='.2f',           
    linewidths=0.5,      
    cbar_kws={'shrink': 0.8},  
    annot_kws={"size": 12},     
    square=True          
)

# Add title and labels for clarity
plt.title('Correlation Matrix of Features', fontsize=16)
plt.xlabel('Features', fontsize=14)
plt.ylabel('Features', fontsize=14)

# Show the plot
plt.show()

## Data Preprocessing

### Train Test Split

We split data into training and testing sets, with 80% of the data in the training set, and 20% in the testing dataset, using random_state=42. We stratified using y because our target data is very imbalanced.

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=random_state)

In [ ]:
print('Training data set size:', X_train.shape[0])
print('Testing data set size:', X_test.shape[0])

### Data Scaling

We scaled the training data using standardization for models such as ANN. This was performed after the train test split to avoid data leakage.

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Training and Evaluation

### Artificial Neural Network Model

#### ANN Training & Hyperparameter Tuning

In [ ]:
# Ignore warnings to make the output cleaner
import warnings
warnings.filterwarnings('ignore')

In [23]:
# Raw data
from sklearn.neural_network import MLPClassifier

y_train = np.ravel(y_train)  # reshaping to avoid warning

hidden_layers = [
    (100,),            # 1 layer with 100 nodes
    (200,),            # 1 layer with 200 nodes
    (100, 50),         # 2 layers: 100 nodes, then 50 nodes
    (200, 100),        # 2 layers: 200 nodes, then 100 nodes
    (100, 50, 25)      # 3 layers: 100 -> 50 -> 25
]

# Creating a grid of hyperparameters
param_grid_ann = {
    'hidden_layer_sizes': hidden_layers,
    'activation': ['relu', 'tanh', 'logistic'],
    'learning_rate_init': [0.005, 0.001],
    'max_iter': [1000],
    'alpha': [0.0005, 0.001, 0.01],
}

# Creating ANN model
ann_model_raw = MLPClassifier(random_state=random_state, early_stopping=True)

# Performing Grid Search to find the best hyperparameters
from sklearn.model_selection import GridSearchCV
grid_search_ann_raw = GridSearchCV(estimator=ann_model_raw, param_grid=param_grid_ann, cv=4, n_jobs=-1, verbose=1, scoring='accuracy')
grid_search_ann_raw.fit(X_train, y_train)

# Best hyperparameters
best_params_ann_raw = grid_search_ann_raw.best_params_
print("Best Hyperparameters:", best_params_ann_raw)

# Best estimator
best_ann_model_raw = grid_search_ann_raw.best_estimator_
y_pred_ann_raw = best_ann_model_raw.predict(X_test)

# Evaluation
from sklearn.metrics import classification_report, confusion_matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_ann_raw))
print("Classification Report:")
print(classification_report(y_test, y_pred_ann_raw))

from sklearn.metrics import accuracy_score
accuracy_ann_raw = accuracy_score(y_test, y_pred_ann_raw)
print(f"Best Mean Accuracy with raw data: {grid_search_ann_raw.best_score_:.4f}")
print(f"Best Accuracy on Test Set with raw data: {accuracy_ann_raw:.4f}")


Fitting 4 folds for each of 90 candidates, totalling 360 fits


Best Hyperparameters: {'activation': 'logistic', 'alpha': 0.01, 'hidden_layer_sizes': (100, 50), 'learning_rate_init': 0.005, 'max_iter': 1000}
Confusion Matrix:
[[  0   0   4   1   1   0   0]
 [  0   0  31  12   0   0   0]
 [  0   0 214 212   2   0   0]
 [  0   0 120 409  38   0   0]
 [  0   0   7 149  60   0   0]
 [  0   0   3  20  16   0   0]
 [  0   0   0   1   0   0   0]]
Classification Report:
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        43
           5       0.56      0.50      0.53       428
           6       0.51      0.72      0.60       567
           7       0.51      0.28      0.36       216
           8       0.00      0.00      0.00        39
           9       0.00      0.00      0.00         1

    accuracy                           0.53      1300
   macro avg       0.23      0.21      0.21      1300
weighted avg       0.49      0.53      0.49      1300



/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
# Scaled data

# Creating ANN model
ann_model_scaled = MLPClassifier(random_state=random_state, early_stopping=True)

# Performing Grid Search to find the best hyperparameters
grid_search_ann_scaled = GridSearchCV(estimator=ann_model_scaled, param_grid=param_grid_ann, cv=4, n_jobs=-1, verbose=1)
grid_search_ann_scaled.fit(X_train_scaled, y_train)

# Best hyperparameters
best_params_ann_scaled = grid_search_ann_scaled.best_params_
print("Best Hyperparameters:", best_params_ann_scaled)

# Best estimator
best_ann_model_scaled = grid_search_ann_scaled.best_estimator_
y_pred_ann_scaled = best_ann_model_scaled.predict(X_test_scaled)

# Evaluation
from sklearn.metrics import classification_report, confusion_matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_ann_scaled))
# print("Classification Report:")
# print(classification_report(y_test, y_pred_ann_scaled))

accuracy_ann_scaled = accuracy_score(y_test, y_pred_ann_scaled)
print(f"Best Mean Accuracy with scaled data: {grid_search_ann_scaled.best_score_:.4f}")
print(f"Best Accuracy on Test Set with scaled data: {accuracy_ann_scaled:.4f}")


Fitting 4 folds for each of 90 candidates, totalling 360 fits
[CV] END activation=relu, alpha=0.0005, hidden_layer_sizes=(100,), learning_rate_init=0.005, max_iter=1000; total time=   0.3s
[CV] END activation=relu, alpha=0.0005, hidden_layer_sizes=(100,), learning_rate_init=0.005, max_iter=1000; total time=   0.3s
[CV] END activation=relu, alpha=0.0005, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000; total time=   0.4s
[CV] END activation=relu, alpha=0.0005, hidden_layer_sizes=(100,), learning_rate_init=0.005, max_iter=1000; total time=   0.4s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100,), learning_rate_init=0.005, max_iter=1000; total time=   0.4s
[CV] END activation=relu, alpha=0.0005, hidden_layer_sizes=(100,), learning_rate_init=0.001, max_iter=1000; total time=   0.5s
[CV] END activation=relu, alpha=0.001, hidden_layer_sizes=(100,), learning_rate_init=0.005, max_iter=1000; total time=   0.4s
[CV] END activation=relu, alpha=0.001, hidden_layer

In [ ]:
from sklearn.decomposition import PCA

for n_component in range(5, 11):

    pca = PCA(n_components=n_component)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Creating ANN model
    ann_model_pca = MLPClassifier(random_state=random_state, early_stopping=True)

    # Performing Grid Search to find the best hyperparameters
    grid_search_ann_pca = GridSearchCV(estimator=ann_model_pca, param_grid=param_grid_ann, cv=4, n_jobs=-1, verbose=1)
    grid_search_ann_pca.fit(X_train_pca, y_train)

    # Best hyperparameters
    best_params_ann_pca = grid_search_ann_pca.best_params_
    print(f"Best Hyperparameters for PCA (n_components={n_component}):", best_params_ann_pca)

    # Best estimator
    best_ann_model_pca = grid_search_ann_pca.best_estimator_
    y_pred_ann_pca = best_ann_model_pca.predict(X_test_pca)

    # Evaluation
    print(f"Confusion Matrix for PCA-transformed data (n_components={n_component}):")
    print(confusion_matrix(y_test, y_pred_ann_pca))
    # print(f"Classification Report fr PCA-transformed data (n_components={n_component}):")
    # print(classification_report(y_test, y_pred_ann_pca))

    print(f"Best Mean Accuracy with PCA (n_components={n_component}): {grid_search_ann_pca.best_score_:.4f}")
    print(f"Best Accuracy on Test Set with PCA (n_components={n_component}): {accuracy_score(y_test, y_pred_ann_pca):.4f}")


     

Fitting 4 folds for each of 90 candidates, totalling 360 fits
Best Hyperparameters for PCA (n_components=5): {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (100, 50, 25), 'learning_rate_init': 0.005, 'max_iter': 1000}
Confusion Matrix for PCA-transformed data (n_components=5):
[[  0   0   4   2   0   0   0]
 [  0   0  28  14   1   0   0]
 [  0   0 219 201   8   0   0]
 [  0   0 131 407  29   0   0]
 [  0   0   9 168  39   0   0]
 [  0   0   4  21  14   0   0]
 [  0   0   0   1   0   0   0]]
Classification Report fr PCA-transformed data (n_components=5):
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        43
           5       0.55      0.51      0.53       428
           6       0.50      0.72      0.59       567
           7       0.43      0.18      0.25       216
           8       0.00      0.00      0.00        39
           9       0.00      0.00      0.00     

/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Hyperparameters for PCA (n_components=6): {'activation': 'tanh', 'alpha': 0.0005, 'hidden_layer_sizes': (100,), 'learning_rate_init': 0.005, 'max_iter': 1000}
Confusion Matrix for PCA-transformed data (n_components=6):
[[  0   0   4   1   1   0   0]
 [  0   0  21  21   1   0   0]
 [  0   0 274 135  19   0   0]
 [  0   0 182 324  61   0   0]
 [  0   0  25 119  72   0   0]
 [  0   0   6  11  22   0   0]
 [  0   0   0   1   0   0   0]]
Classification Report fr PCA-transformed data (n_components=6):
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        43
           5       0.54      0.64      0.58       428
           6       0.53      0.57      0.55       567
           7       0.41      0.33      0.37       216
           8       0.00      0.00      0.00        39
           9       0.00      0.00      0.00         1

    accuracy                           0.52      1300
   mac

/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Hyperparameters for PCA (n_components=7): {'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (100, 50, 25), 'learning_rate_init': 0.005, 'max_iter': 1000}
Confusion Matrix for PCA-transformed data (n_components=7):
[[  0   0   4   2   0   0   0]
 [  0   0  22  21   0   0   0]
 [  0   0 132 288   8   0   0]
 [  0   0  64 472  31   0   0]
 [  0   0   3 184  29   0   0]
 [  0   0   1  26  12   0   0]
 [  0   0   0   1   0   0   0]]
Classification Report fr PCA-transformed data (n_components=7):
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        43
           5       0.58      0.31      0.40       428
           6       0.47      0.83      0.60       567
           7       0.36      0.13      0.20       216
           8       0.00      0.00      0.00        39
           9       0.00      0.00      0.00         1

    accuracy                           0.49      1300
 

...

/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Hyperparameters for PCA (n_components=9): {'activation': 'relu', 'alpha': 0.0005, 'hidden_layer_sizes': (100, 50, 25), 'learning_rate_init': 0.005, 'max_iter': 1000}
Confusion Matrix for PCA-transformed data (n_components=9):
[[  0   0   3   1   2   0   0]
 [  1   0  27  12   3   0   0]
 [  0   0 205 197  26   0   0]
 [  0   0  95 349 123   0   0]
 [  0   0   4 101 111   0   0]
 [  0   0   1  14  24   0   0]
 [  0   0   0   0   1   0   0]]
Classification Report fr PCA-transformed data (n_components=9):
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.00      0.00      0.00        43
           5       0.61      0.48      0.54       428
           6       0.52      0.62      0.56       567
           7       0.38      0.51      0.44       216
           8       0.00      0.00      0.00        39
           9       0.00      0.00      0.00         1

    accuracy                           0.51      1300

/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best Hyperparameters for PCA (n_components=10): {'activation': 'tanh', 'alpha': 0.01, 'hidden_layer_sizes': (100, 50), 'learning_rate_init': 0.005, 'max_iter': 1000}
Confusion Matrix for PCA-transformed data (n_components=10):
[[  0   0   4   1   1   0   0]
 [  0   2  26  15   0   0   0]
 [  0   0 223 194  11   0   0]
 [  0   1 118 388  59   1   0]
 [  0   0   7 133  75   1   0]
 [  0   0   5  14  17   3   0]
 [  0   0   0   1   0   0   0]]
Classification Report fr PCA-transformed data (n_components=10):
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.67      0.05      0.09        43
           5       0.58      0.52      0.55       428
           6       0.52      0.68      0.59       567
           7       0.46      0.35      0.40       216
           8       0.60      0.08      0.14        39
           9       0.00      0.00      0.00         1

    accuracy                           0.53      1300
  

/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/honghao/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



### K-Nearest Neighbors (KNN) Model

In [5]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


#### kNN Training

In [ ]:
knn_classifier = KNeighborsClassifier()
knn_classifier.fit(X_train, y_train.iloc[:, 0].values.ravel()) # there is some error for pandas datafram
knn_pred = knn_classifier.predict(X_test)
print("K-Nearest Neighbors Classifier")
print("Confusion Matrix:\n", confusion_matrix(y_test, knn_pred))
print("\nClassification Report:\n", classification_report(y_test, knn_pred, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, knn_pred))

#### Fine tuning for KNN

In [21]:
# Raw data

#fine tuning using Gridsearch
param_grid_knn = { 'n_neighbors': [3, 5, 7, 9, 11, 13], 'weights': ['uniform', 'distance'], 'p': [1, 2] }
#the 1 and 2 are different types of distance, manhattan or euclidean
grid_search_knn = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid_knn,
    cv=4,
    n_jobs=-1,
    #verbose=2
)
grid_search_knn.fit(X_train, y_train.iloc[:, 0].values.ravel())
best_knn = grid_search_knn.best_estimator_
print("Best Parameters for KNN:", grid_search_knn.best_params_)
y_best_pred_knn = best_knn.predict(X_test)
print("Confusion Matrix:",confusion_matrix(y_test, y_best_pred_knn))
print("\nClassification Report:\n", classification_report(y_test, y_best_pred_knn, zero_division=0))

print(f"Best Mean CV Accuracy with StandardScaler: {grid_search_knn.best_score_:.4f}")
print(f"Best Accuracy on Test Set with StandardScaler: {accuracy_score(y_test, grid_search_knn.predict(X_test)):.4f}")


Best Parameters for KNN: {'n_neighbors': 13, 'p': 1, 'weights': 'distance'}
Confusion Matrix: [[  0   0   2   4   0   0   0]
 [  0   5  23  14   1   0   0]
 [  0   3 269 138  16   2   0]
 [  0   0 108 416  41   2   0]
 [  0   0  20  89 105   2   0]
 [  0   0   3  17   6  13   0]
 [  0   0   0   1   0   0   0]]

Classification Report:
               precision    recall  f1-score   support

           3       0.00      0.00      0.00         6
           4       0.62      0.12      0.20        43
           5       0.63      0.63      0.63       428
           6       0.61      0.73      0.67       567
           7       0.62      0.49      0.55       216
           8       0.68      0.33      0.45        39
           9       0.00      0.00      0.00         1

    accuracy                           0.62      1300
   macro avg       0.45      0.33      0.36      1300
weighted avg       0.62      0.62      0.61      1300

Best Mean CV Accuracy with StandardScaler: 0.5905
Best Accuracy on

In [8]:
# Scaled data

grid_search_knn_scaled = GridSearchCV(
    estimator=KNeighborsClassifier(),
    param_grid=param_grid_knn,
    cv=4,
    n_jobs=-1,
    #verbose=2
)
grid_search_knn_scaled.fit(X_train_scaled, y_train.iloc[:, 0].values.ravel())
best_knn_scaled = grid_search_knn_scaled.best_estimator_
print(f"Best Parameters for SVM (C and kernel) with StandardScaler:", grid_search_knn_scaled.best_params_)
print(f"Best Mean CV Accuracy with StandardScaler: {grid_search_knn_scaled.best_score_:.4f}")
print(f"Best Accuracy on Test Set with StandardScaler: {accuracy_score(y_test, best_knn_scaled.predict(X_test_scaled)):.4f}")

Best Parameters for SVM (C and kernel) with StandardScaler: {'n_neighbors': 11, 'p': 1, 'weights': 'distance'}
Best Mean CV Accuracy with StandardScaler: 0.6456
Best Accuracy on Test Set with StandardScaler: 0.6554


In [20]:
# PCA data
from sklearn.decomposition import PCA 

for n_component in range(5, 11):
    
    pca = PCA(n_components=n_component)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)

    # Performing Grid Search to find the best hyperparameters
    grid_search_knn_pca = GridSearchCV(
        estimator=KNeighborsClassifier(),
        param_grid=param_grid_knn,
        cv=4,
        n_jobs=-1,
        #verbose=1
    )
    grid_search_knn_pca.fit(X_train_pca, y_train.iloc[:, 0].values.ravel())

    # Best hyperparameters
    best_params_knn_pca = grid_search_knn_pca.best_params_
    print(f"Best Hyperparameters for PCA (n_components={n_component}):", best_params_knn_pca)

    # Best estimator
    best_knn_model_pca = grid_search_knn_pca.best_estimator_
    y_pred_knn_pca = best_knn_model_pca.predict(X_test_pca)

    # Evaluation
    print(f"Confusion Matrix for PCA-transformed data (n_components={n_component}):")
    print(confusion_matrix(y_test, y_pred_knn_pca))
    #print(f"Classification Report fr PCA-transformed data (n_components={n_component}):")
    #print(classification_report(y_test, y_pred_knn_pca, zero_division=1))

    print(f"Best Mean Accuracy with PCA (n_components={n_component}): {grid_search_knn_pca.best_score_:.4f}")
    print(f"Best Accuracy on Test Set with PCA (n_components={n_component}): {accuracy_score(y_test, y_pred_knn_pca):.4f}")

Best Hyperparameters for PCA (n_components=5): {'n_neighbors': 13, 'p': 1, 'weights': 'distance'}
Confusion Matrix for PCA-transformed data (n_components=5):
[[  0   0   2   4   0   0   0]
 [  0   5  21  15   2   0   0]
 [  0   2 265 144  17   0   0]
 [  0   1  99 422  43   2   0]
 [  0   0  21  89 104   2   0]
 [  0   0   2  19   5  13   0]
 [  0   0   0   1   0   0   0]]
Best Mean Accuracy with PCA (n_components=5): 0.5830
Best Accuracy on Test Set with PCA (n_components=5): 0.6223
Best Hyperparameters for PCA (n_components=6): {'n_neighbors': 13, 'p': 1, 'weights': 'distance'}
Confusion Matrix for PCA-transformed data (n_components=6):
[[  0   0   2   4   0   0   0]
 [  0   5  22  14   2   0   0]
 [  0   1 267 144  16   0   0]
 [  0   1 101 419  44   2   0]
 [  0   0  19  90 105   2   0]
 [  0   0   3  17   5  14   0]
 [  0   0   0   1   0   0   0]]
Best Mean Accuracy with PCA (n_components=6): 0.5836
Best Accuracy on Test Set with PCA (n_components=6): 0.6231
Best Hyperparameters f

### Support Vector Machine

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

param_grid_svm = {
    'C': [0.05, 0.1, 1, 5, 10, 20, 50],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

grid_search_svm = GridSearchCV(
    estimator=SVC(),
    param_grid=param_grid_svm,
    cv=4,
    n_jobs=-1,
    scoring='accuracy',
    return_train_score=True,
    verbose=2,
)

grid_search_svm.fit(X_train, y_train.iloc[:, 0].values.ravel())
best_svm = grid_search_svm.best_estimator_
print("Best Parameters for SVM:", grid_search_svm.best_params_)
y_best_pred_svm = best_svm.predict(X_test)
print("Confusion Matrix:",confusion_matrix(y_test, y_best_pred_svm))
print("\nClassification Report:\n", classification_report(y_test, y_best_pred_svm, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, y_best_pred_svm))
print(f"Best Mean CV Accuracy: {grid_search_svm.best_score_:.4f}")

heatmap = sns.heatmap(
    grid_search_svm.cv_results_['mean_test_score'].reshape(len(param_grid_svm['C']), len(param_grid_svm['kernel'])),
    annot=True,
    fmt=".3f",
    cmap="YlGnBu",
    xticklabels=param_grid_svm['kernel'],
    yticklabels=param_grid_svm['C']
)
plt.title("SVM Hyperparameter Tuning Heatmap")
plt.xlabel("Kernel")
plt.ylabel("C Value")
plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

n_components = 7  # Set the number of components for PCA
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

param_grid_svm = {
        'C': [0.05, 0.1, 1, 5, 10, 20, 50],
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
}

grid_search_svm = GridSearchCV(
    estimator=SVC(),
    param_grid=param_grid_svm,
    cv=4,
    n_jobs=-1,
    scoring='accuracy',
    return_train_score=True,
    verbose=2,
)

grid_search_svm.fit(X_train_pca, y_train.iloc[:, 0].values.ravel())

best_svm = grid_search_svm.best_estimator_
best_params = grid_search_svm.best_params_

print(f"Best Parameters for SVM (C and kernel) with PCA (n_components={n_components}):", best_params)
print(f"Best Mean CV Accuracy with PCA (n_components={n_components}): {grid_search_svm.best_score_:.4f}")
print(f"Best Accuracy on Test Set with PCA (n_components={n_components}): {accuracy_score(y_test, best_svm.predict(X_test_pca)):.4f}")

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

grid_search_svm = GridSearchCV(
    estimator=SVC(),
    param_grid=param_grid_svm,
    cv=4,
    n_jobs=-1,
    scoring='accuracy',
    return_train_score=True,
    verbose=2,
)

grid_search_svm.fit(X_train_scaled, y_train.iloc[:, 0].values.ravel())
best_svm = grid_search_svm.best_estimator_
best_params = grid_search_svm.best_params_

print(f"Best Parameters for SVM (C and kernel) with StandardScaler:", best_params)
print(f"Best Mean CV Accuracy with StandardScaler: {grid_search_svm.best_score_:.4f}")
print(f"Best Accuracy on Test Set with StandardScaler: {accuracy_score(y_test, best_svm.predict(X_test_scaled)):.4f}")

### Scores

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
print("Best Parameters:", grid_search.best_params_)

best_svm_classifier = SVC(kernel=grid_search.best_params_['kernel'], C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'], random_state=random_state)
best_svm_classifier.fit(X_train_scaled, y_train.iloc[:, 0].values.ravel())
y_pred_best_svm = best_svm_classifier.predict(X_test_scaled)
print("Best SVM Classifier")
# print("\nClassification Report:\n", classification_report(y_test, y_pred_best_svm, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred_best_svm))
print("f1 Score:", f1_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Precision Score:", precision_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Recall Score:", recall_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best_svm))

best_svm_classifier.fit(X_train, y_train.iloc[:, 0].values.ravel())
y_pred_best_svm = best_svm_classifier.predict(X_test)
print("Best SVM Classifier")
# print("\nClassification Report:\n", classification_report(y_test, y_pred_best_svm, zero_division=0))
print("\nAccuracy Score:", accuracy_score(y_test, y_pred_best_svm))
print("f1 Score:", f1_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Precision Score:", precision_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Recall Score:", recall_score(y_test, y_pred_best_svm, average='weighted', zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_best_svm))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

knn_classifier = KNeighborsClassifier(n_neighbors=13, weights='distance', p=1)
knn_classifier.fit(X_train, y_train.iloc[:, 0].values.ravel())
y_pred_knn_raw = knn_classifier.predict(X_test)

print("raw KNN")
print("Accuracy Score:", accuracy_score(y_test, y_pred_knn_raw))
print("f1 Score:", f1_score(y_test, y_pred_knn_raw, average='weighted', zero_division=0))
print("Precision Score:", precision_score(y_test, y_pred_knn_raw, average='weighted', zero_division=0))
print("Recall Score:", recall_score(y_test, y_pred_knn_raw, average='weighted', zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn_raw))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn_raw, zero_division=0))

In [ ]:
from sklearn.preprocessing import StandardScaler

#standardized
scaler = StandardScaler()
X_train_scaled_knn = scaler.fit_transform(X_train)
X_test_scaled_knn = scaler.transform(X_test)

knn_classifier_scaled = KNeighborsClassifier(n_neighbors=13, weights='distance', p=1)
knn_classifier_scaled.fit(X_train_scaled_knn, y_train.iloc[:, 0].values.ravel())
y_pred_knn_scaled = knn_classifier_scaled.predict(X_test_scaled_knn)

print("scaled KNN")
print("Accuracy Score:", accuracy_score(y_test, y_pred_knn_scaled))
print("f1 Score:", f1_score(y_test, y_pred_knn_scaled, average='weighted', zero_division=0))
print("Precision Score:", precision_score(y_test, y_pred_knn_scaled, average='weighted', zero_division=0))
print("Recall Score:", recall_score(y_test, y_pred_knn_scaled, average='weighted', zero_division=0))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn_scaled))
print("\nClassification Report:\n", classification_report(y_test, y_pred_knn_scaled, zero_division=0))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Data from Table 5 and Table 6
models = ['ANN', 'KNN', 'SVM']
metrics = ['Accuracy', 'F1', 'Precision', 'Recall']

# Raw input scores
raw_scores = {
    'Accuracy': [0.5108, 0.6215, 0.6169],
    'F1': [0.54, 0.6095, 0.5812],
    'Precision': [0.53, 0.62, 0.73],
    'Recall': [0.57, 0.6215, 0.6169]
}

# Standardized input scores
standardized_scores = {
    'Accuracy': [0.6208, 0.6623, 0.6538],
    'F1': [0.46, 0.6508, 0.6450],
    'Precision': [0.49, 0.658, 0.6481],
    'Recall': [0.51, 0.6623, 0.6538]
}

x = np.arange(len(models))
width = 0.35

# Plotting each metric separately
for metric in metrics:
    plt.figure(figsize=(8, 5))
    plt.bar(x - width/2, raw_scores[metric], width, label='Raw')
    plt.bar(x + width/2, standardized_scores[metric], width, label='Standardized')
    plt.xticks(x, models)
    plt.ylabel(metric)
    plt.title(f'{metric} Comparison: Raw vs Standardized Input')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import cycle
import numpy as np


classes = sorted(y_train.iloc[:, 0].unique())
n_classes = len(classes)
y_test_binarized = label_binarize(y_test, classes=classes)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    'KNN': OneVsRestClassifier(KNeighborsClassifier(n_neighbors=13, weights='distance', p=1)),
    'SVM': OneVsRestClassifier(SVC(C=50, kernel='rbf', gamma=1, probability=True)),
    'ANN': OneVsRestClassifier(MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42))
}

colors = cycle(['darkorange', 'blue', 'green'])
plt.figure(figsize=(10, 8))

for (name, model), color in zip(models.items(), colors):
    model.fit(X_train_scaled, label_binarize(y_train.iloc[:, 0], classes=classes))
    y_score = model.predict_proba(X_test_scaled)

  
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_binarized.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    
    plt.plot(fpr["micro"], tpr["micro"],
             label=f'{name} (AUC = {roc_auc["micro"]:.2f})', color=color)

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Micro-Averaged ROC Curve for Multiclass Models')
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()